In [3]:
import matplotlib.pyplot as plt  
# iscrtavanje slika i plotova unutar samog browsera
%matplotlib inline 

import matplotlib.pylab as pylab
# prikaz vecih slika 
pylab.rcParams['figure.figsize'] = 21,15

import numpy as np
import cv2 # OpenCV biblioteka

def show_in_window_and_below(img, below=True):
    if (below):
        plt.imshow(img, 'gray')
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# current image extension
ext = '.JPG'

In [4]:
# runs calculations
def calculate_runs(img):
    runs = [[1] for x in xrange(img.shape[1])] # each column starts with 1 black pixel
    black_runs_flat, white_runs_flat = [], []
    for col in xrange(img.shape[1]): # iterate through all columns
        img[0,col] = 0 # PAINT THE FIRST PIXEL IN PREDEFINED COLOR, to make all columns start the same
        run_index = 0 # start the run
        for row in xrange(1, img.shape[0]): # for each pixel/row in current column
            if (img[row-1 ,col] != img[row, col]):  # if they are not the same,
                # memorize the old run in corresponding array
                if (run_index % 2 == 0): # black run
                    black_runs_flat.append(runs[col][run_index])
                else:
                    white_runs_flat.append(runs[col][run_index])
                # start a new run
                run_index += 1
                runs[col].append(0)
            runs[col][run_index] += 1     # add a pixel to the current run
        # the column ended ==> save the last run for the ended column
        if (run_index % 2 == 0): # black run
            black_runs_flat.append(runs[col][run_index])
        else:
            white_runs_flat.append(runs[col][run_index])
    return runs, black_runs_flat, white_runs_flat

In [5]:
# line thickness and spacings (black and white runs analysis)
from collections import Counter

def calculate_line_thickness(black_flat):
    num_top = 4
    black_count = Counter(black_flat) # Counter({1: 3, 8: 1, 3: 1, ...})
    m_c_black = black_count.most_common(num_top)
    m_c_black1, m_c_black2 = m_c_black[0][0], m_c_black[1][0]
    print 'Top', num_top, 'most common black runs:', m_c_black
    if (m_c_black1*3 < m_c_black2): # kind of a sanity check
        line_thickness = m_c_black1
    else:
        line_thickness = (m_c_black1 + m_c_black2) / 2.
    print '>>> line thickness:  ', line_thickness
    return line_thickness

def calculate_line_spacing(white_flat, image_height):
    num_top = 4
    white_count = Counter(white_flat) # print white_count.most_common(50)
    m_c_white = white_count.most_common(num_top)
    m_c_white1, m_c_white2 = m_c_white[0][0], m_c_white[1][0]
    print 'Top', num_top, 'most common white runs', m_c_white
    
    if (m_c_white1 > image_height*0.5): # sanity check
        line_spacing = m_c_white2
    else:
        if (m_c_white2 > image_height*0.5):
            line_spacing = m_c_white1
        else:
            line_spacing = (m_c_white1 + m_c_white2) / 2.
    print 'line spacing: ', line_spacing
    return line_spacing

In [6]:
def remove_staff_lines(img, runs, line_height, staff_thickness_multiplier):
    # copy the image.. python is pass-by-object-reference so it is necessary!
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    #edit the image
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            if (r % 2 == 0): # black runs # every black run longer than 2 * line_height is deleted/whitened
                if (run_length < line_height * staff_thickness_multiplier):
                    # ++ AKO JE SLJEDEĆI/PRETHODNI %% BIJELI %% RUN = VISINA PRAZNINE +-1
                    # ++ AKO JE SLJEDEĆI/PRETHODNI %%  CRNI  %% RUN = VISINA LINIJE +-1
                    p[cumulative:cumulative + run_length, c] = [255]*(run_length)
            #else: # white runs
            #    do something maybe ?
            cumulative += run_length
    return p

In [7]:
#########
# USE:
#    = in lines-only image, to LOCATE the lines AND/OR check if there is a line on the current location/run
#        - easier to find lines, since there are no other elements
#        - takes more time, since we need to generate the lines-only image,
#           ,BUT THAT IS NOT A PROBLEM SINCE WE WILL NEED IT TO LOCATE THE LINES
#
#    = in binary image, to check if there is a line on the current location/run
#
# da se utvrde linije treba samo odrediti visinu, a ne 
# TIP: mozda za svaki linijski sistem (ili cak liniju) cuvati vise x, koordinata,
#   recimo na svaku petinu sirine slike provjeravati lokacije
#   linijskih sistema (ili pojedinacnih linija) na vise mijesta u slici:
#       |       |        |      |
#       V       V        V      V
# -----..____..--------------------  <== curved line, others are ok
# --------------------------------- 
# ---------------------------------
# ---------------------------------
# ---------------------------------
######
# x je niz od onoliko crnih piksela koliko je prosjecna debljina linije (+-1 ili 2)
# x = [0] * (int(thickness)-1)
# y je niz od onoliko crnih piksela koliko je prosjecna debljina linije + 2 ili 3 ||| INT!
#   ~ pikseli iz y niza pocinju od posljednje tacke posmatrane linije (tj.kandidata za liniju)
# y = current_pos + int(spacing * 0.9 ili 0.8)
# print x_in_y(x, y)
###
def x_in_y(x, y):
    try:
        x_len = len(x)
    except TypeError:
        x_len = 1
        x = type(y)((x,))

    for i in xrange(len(y)):
        if (y[i : i+x_len] == x):
            return True
    return False

# a = [0,0,1,1,1,0,0,0,0]
# b = [1,1,1]
# b = 0 # works also
# print x_in_y(b, a)

In [8]:
# # # # # # # # # # # # # # # # # # # # # # # # # # #
# ALG 2 - look at white runs around the line candidate
# 2*d . . . 1*d . . . X . . . 1*d . . . 2*d
#  0         1        -        2         3
# In order for X to be whitened (erased):
#

def spacing_is_ok(run_length, line_spacing):
#     print '>>> spacing_is_ok <<< run_length: {}, line_spacing: {}'.format(run_length, line_spacing)
    return (run_length < (line_spacing * 1.15)) and (run_length > (line_spacing * 0.85))

def run_has_lines_up_or_down(runs, current_index, line_thickness, line_spacing):
    '''check if there is a line on `line_spacing` above or below the current index '''
    lines = [255,255,255,255]
    return 1

def rm_staff_lines_up_down_neighbours(img, runs, line_thickness, line_spacing, staff_thickness_multiplier):
    '''Removes staff lines by looking at every black run's neighbours, above and below.'''
    # Original image is being changed + Python passes by obj-ref, so it's necessary
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            deleted = False # did we delete the run
            if (r % 2 == 0): # check every black run, shorter than `thickness * multiplier`
                if (run_length < line_thickness * staff_thickness_multiplier):
                    # this is not the last run === check the run AFTER this one
                    if (r + 1 < (len(runs[c]))):
                        if spacing_is_ok(runs[c][r+1], line_spacing): # we found a line - delete it
                            p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                            deleted = True
                        # else: # more conditions to add.. like..
                                # check if there is a black run above or below the current one,
                                # on distance that is equal to line_spacing +-1,
                                # whose lenght is = line_thickness +-1
                            
                    # this is not the first run === check the run BEFORE this one
                    if (not deleted and (r - 1 >= 0)):
                        if spacing_is_ok(runs[c][r-1], line_spacing): # we found a line - delete it
                            p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                            deleted = True
                    if (not deleted and run_has_lines_up_or_down()):
                        
            cumulative += run_length # !!!
    return p

IndentationError: expected an indented block (<ipython-input-8-2ea6a38f66a6>, line 46)

In [9]:
# # # # # # # # # # # # # # # # # # # # # # # # # # #
# ALG 3 - compare neighbour points on distance `d`,
# to the left and right, from the observed point X:
# 2*d . . . 1*d . . . X . . . 1*d . . . 2*d
#  0         1        -        2         3
# In order for X to be whitened (erased):
#   - points[1] and points[2] should be black, or
#   - points[0] and points[1] should be black, or
#   - points[2] and points[3] should be black.

def initialize_points(c, distance, img, compare_point, runs):
    '''Gets the neighbours located '''
    points = [255,255,255,255] # initially, pixels are white (maybe -1 if not accessible)
    # 2*left , 1*left , 1*right, 2*right
    if (c - distance >= 0): # we can get the 1*left pixel
        points[1] = img[compare_point, c - distance]
        if (c - 2*distance >= 0): # we can get the 2*left pixel
            points[0] = img[compare_point, c - 2*distance]
    if (c + distance < len(runs)): # we can get the right pixel
        points[2] = img[compare_point, c + distance]
        if (c + 2*distance < len(runs)):
            points[3] = img[compare_point, c + 2*distance]
    return points

def rm_staff_lines_side_neighbours(img, runs, thickness, spacing, thickness_mul, distance):
    if distance > len(runs)/2:
        print 'WHOA! Distance is: {} and there are only {} columns'.format(distance, len(runs))
        return img
    int_thickness = int(thickness)+1
    p = img.copy() # we will erase some lines, so copy the image
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    for c in xrange(len(runs)): # for every column
        cumulative = 0 # number of passed pixels
        for r in xrange(len(runs[c])):
            run_length = runs[c][r]
            if (r % 2 == 0):# for every black run
                if (run_length < thickness * thickness_mul):
                    p[cumulative:cumulative + run_length, c] = [255]*(run_length)
                    cumulative += run_length # !!!
                    continue
                else:
                    pixels_to_remove = int_thickness
                compare_point = cumulative + pixels_to_remove/2 # + run_length/2 # maybe later
                # 4 values of neighbour pixels, some of them must be black,
                #   for deletion of the observed pixel to happen.
                points = initialize_points(c, distance, img, compare_point, runs)
                if (points[1] == 0):
                    if (points[2] == 0):
                        p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
                    else: # out of bounds or white # !! !!  ASSUMPTION !!  !!!
                    # if (points[2] == -1): # more robust? Needs points=[-1,-1,-1,-1]
                        # two to the left is black?
                        if (points[0] == 0):
                            p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
                        # else: NIJE LINIJA :D
                # no black point on the left
                else:
                    if (points[2] == 0 and points[3] == 0):
                        p[cumulative: cumulative + pixels_to_remove, c] = [255]*pixels_to_remove
            cumulative += run_length # !!!
    # return the new, processed image
    return p

In [79]:
def parse_image(img, threshold_type, block_size, c_value, staff_thickness_multiplier, params=[]):
    t_t, b_s, s_t_m = threshold_type, block_size, staff_thickness_multiplier
    print('=========\nthreshold_type: {}, block_size: {}, c_value: {}, staff_thickness_multiplier: {}'.format(t_t, b_s, c_value, s_t_m))
    img_ada = cv2.adaptiveThreshold(img, 255, threshold_type, cv2.THRESH_BINARY, block_size, c_value)
    
    dilate_kernel = np.ones((1,30), dtype=np.int) # np.ones((kernel_w, kernel_h), dtype=np.int);
    # staff lines LOCATIONS, along with lines-only image
    lines_only_img, locations = cv2.dilate(img_ada, kernel, iterations=1)
    
    # runs calculation
    runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada)
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # needs image height
    result = rm_staff_lines_up_down_neighbours(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier)
#     distance = int(line_spacing * 0.5)
#     result = rm_staff_lines_side_neighbours(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier, distance)
#     result = rm_s(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier, distance)
    
    cv2.imwrite('./images/dataset/run_X/params_'+str(t_t)+'_'+str(b_s)+'_'+str(c_value)+'_'+str(s_t_m)+ext, result)
    #cv2.imwrite('./images/dataset/run_Y/params_ORIGINAL.jpg', img_ada)

img = cv2.imread('images/dataset/muzikanti'+ext, 0) #  0 -->  read as grayscale
# parse_image(img, cv2.ADAPTIVE_THRESH_MEAN_C, 33, 35, 1.5)

In [249]:
# param values
threshold_types = [cv2.ADAPTIVE_THRESH_MEAN_C]# , cv2.ADAPTIVE_THRESH_GAUSSIAN_C # const values are 0 and 1
block_sizes = [33,35,37,39,43]# [11, 19, 27, 35, 43] # [11,15,19,23,27,31,35, 39, 43]
c_values = [29,33,35,37] # [17,19,21] # [11, 19, 27, 35, 43] # [11,15,19,23,27,31,35, 39, 43]
staff_thickness_multipliers = [1.3, 1.5] # [1., 1.5, 2., 2.5]

In [68]:
for thresh_type in threshold_types:
    for block_size in block_sizes:
        for c_val in c_values:
            for s_t_mul in staff_thickness_multipliers:
                parse_image(img, thresh_type, block_size, c_val, s_t_mul)

threshold_type: 0, block_size: 33, c_value: 29, staff_thickness_multiplier: 1.3
Top two most common black runs: [(4, 36510), (3, 33167)]
>>> line height:   3.5
Top two most common white runs [(21, 24794), (20, 15403)]
space height:  20.5
threshold_type: 0, block_size: 33, c_value: 29, staff_thickness_multiplier: 1.5
Top two most common black runs: [(4, 36510), (3, 33167)]
>>> line height:   3.5
Top two most common white runs [(21, 24794), (20, 15403)]
space height:  20.5
threshold_type: 0, block_size: 33, c_value: 33, staff_thickness_multiplier: 1.3
Top two most common black runs: [(3, 38242), (4, 31349)]
>>> line height:   3.5
Top two most common white runs [(21, 26952), (20, 12360)]
space height:  20.5
threshold_type: 0, block_size: 33, c_value: 33, staff_thickness_multiplier: 1.5
Top two most common black runs: [(3, 38242), (4, 31349)]
>>> line height:   3.5
Top two most common white runs [(21, 26952), (20, 12360)]
space height:  20.5
threshold_type: 0, block_size: 33, c_value: 35, 

Top two most common black runs: [(4, 36534), (3, 32881)]
>>> line height:   3.5
Top two most common white runs [(21, 25559), (20, 15173)]
space height:  20.5
threshold_type: 0, block_size: 43, c_value: 33, staff_thickness_multiplier: 1.3
Top two most common black runs: [(3, 37651), (4, 31786)]
>>> line height:   3.5
Top two most common white runs [(21, 27641), (20, 12117)]
space height:  20.5
threshold_type: 0, block_size: 43, c_value: 33, staff_thickness_multiplier: 1.5
Top two most common black runs: [(3, 37651), (4, 31786)]
>>> line height:   3.5
Top two most common white runs [(21, 27641), (20, 12117)]
space height:  20.5
threshold_type: 0, block_size: 43, c_value: 35, staff_thickness_multiplier: 1.3
Top two most common black runs: [(3, 39893), (4, 29339)]
>>> line height:   3.5
Top two most common white runs [(21, 28482), (20, 10654)]
space height:  20.5
threshold_type: 0, block_size: 43, c_value: 35, staff_thickness_multiplier: 1.5
Top two most common black runs: [(3, 39893), (4,

In [91]:
praznina = 20
# ln - linija  || pr - praznina || C- crno-ne-linija || B- bijelo-ne-praznina
#     ln1,pr1,ln2,pr2,C,pr2,ln3,pr3
col_runs = [4, 20, 4, 10, 3, 7, 4, 21, 4, 21] # run-ovi u jednoj koloni
index = 4 # sad smo kod spornog crnog run-a
# prethodni bijeli run sirok kao praznina AND razlika izmedju 
is_line = False
for i in range(0, len(col_runs), 2):
    print 'INDEKS: ', i
    # nije posljednji run === gledamo naredni run
    if (i < len(col_runs) - 1):
        run_after = col_runs[i+1]
        if (run_after < praznina * 1.15) and (run_after > praznina * 0.85):
            print '\tLINIJA'
            continue # nema potrebe da provjeravamo prazninu iznad
    # nije prvi run === gledamo prehodni run
    if (i > 0):
        run_before = col_runs[i-1]
        if (run_before < praznina * 1.15) and (run_before > praznina* 0.85): #abs(col_runs[index-1] - praznina) < praznina * 0.1)
            print '\tLINIJA'

# index = 8 # ovo je stvarno linija
# 20*0.85

INDEKS:  0
	LINIJA
INDEKS:  2
	LINIJA
INDEKS:  4
INDEKS:  6
	LINIJA
INDEKS:  8
	LINIJA


In [ ]:
kernel_w, kernel_h = 1, 30
kernel_w, kernel_h = 30, 1
kernel_w, kernel_h = 15, 3
kernel_w, kernel_h = 10, 10
kernel_w, kernel_h = 5, 5
kernel_w, kernel_h = 3, 3
kernel = np.ones((kernel_w, kernel_h), dtype=np.int);

# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5))
# print 'kernel', kernel

img_edited1 = cv2.imread('./images/dataset/params_0_33_35_1.5.JPG')
img_edited2 = cv2.imread('./images/dataset/params_0_35_19_1.5.JPG')

dilated1 = cv2.dilate(img_edited1, kernel, iterations=1)
cv2.imwrite('./images/dataset/dilated_33_35_kernel_{}_{}.jpg'.format(kernel_w, kernel_h), dilated1)
dilated2 = cv2.dilate(img_edited2, kernel, iterations=1)
cv2.imwrite('./images/dataset/dilated_35_19_kernel_{}_{}.jpg'.format(kernel_w, kernel_h), dilated2)

eroded1 = cv2.erode(dilated1, kernel, iterations=1)
cv2.imwrite('./images/dataset/er_dil_35_19_kernel_{}_{}.jpg'.format(kernel_w, kernel_h), eroded1)

eroded2 = cv2.erode(dilated2, kernel, iterations=1)
cv2.imwrite('./images/dataset/er_dil_33_35_kernel_{}_{}.jpg'.format(kernel_w, kernel_h), eroded2)

In [65]:
kernel_w, kernel_h = 10,10
dil = cv2.dilate(img_edited2, kernel, iterations=1)
er = cv2.erode(dil, kernel, iterations=1)
cv2.imwrite('./images/dataset/er_dil_w5_h2.jpg', er)

True

In [67]:
kernel_w, kernel_h = 10,10
er = cv2.erode(img_edited2, kernel, iterations=1)
dil = cv2.dilate(er, kernel, iterations=1)
cv2.imwrite('./images/dataset/dil_er_w{}_h{}.jpg'.format(kernel_w, kernel_h), dil)

True

In [78]:
# Test various kernel dimensions on  binarized images with
# morph. operations: erosion, dilation, opening  and  closing
kernel_widths = [2, 3, 5, 7]
kernel_heights = [2, 3, 5, 7]

block_sizes = [51, 51]
c_values = [35, 45]
for i in range(2):
    img_path = './images/adaptive_params_testing/t_0_b_{}_c_{}.jpg'.format(block_sizes[i], c_values[i])
    img = cv2.imread(img_path, 0) # grayscale! (actually binarized, but format is the same)
    for kernel_width in kernel_widths:
        for kernel_height in kernel_heights:
            kernel = np.ones((kernel_width, kernel_height), dtype=np.int)
            eroded = cv2.erode(img, kernel, iterations=1)
            dilated = cv2.dilate(img, kernel, iterations=1)
            er_b4_dil = cv2.dilate(er, kernel, iterations=1)
            dil_b4_er = cv2.erode(dil, kernel, iterations=1)
            
            cv2.imwrite('./images/kernel_2/ER_b_{}_c_{}_kw_{}_kh_{}.jpg'\
                        .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  eroded)
            cv2.imwrite('./images/kernel_2/DIL_b_{}_c_{}_kw_{}_kh_{}.jpg'\
                        .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  dilated)
            cv2.imwrite('./images/kernel_2/ER_B4_DIL_b_{}_c_{}_kw_{}_kh_{}.jpg'\
                        .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  er_b4_dil)
            cv2.imwrite('./images/kernel_2/DIL_B4_ER_b_{}_c_{}_kw_{}_kh_{}.jpg'\
                        .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  dil_b4_er)
# end

In [26]:
block_sizes = [51]#, 51] #  ]#
c_values = [35]#, 45]    #  ]#
thresh = cv2.ADAPTIVE_THRESH_MEAN_C
method = cv2.THRESH_BINARY
img = cv2.imread('images/dataset/muzikanti'+ext, 0) #  0 => read as grayscale

for i in range(len(block_sizes)):
    block = block_sizes[i]
    c = c_values[i]
    img = cv2.adaptiveThreshold(img, 255, thresh, method, block, c)
    
    k_w = 3
    k_h = 1
    kernel = np.ones((k_w, k_h), dtype=np.int)
#     img_er = cv2.erode(img, kernel)
#     cv2.imwrite('./images/e/51_31_k{}_{}.jpg'.format(k_w, k_h), img_er)
    
#     img = img_er # !!! # da ne mijenjam svuda
    
    # img.shape # [ VISINA, SIRINA ]
    
    runs, black_runs_flat, white_runs_flat = calculate_runs(img) # runs calculation, for thickness and spacing
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img.shape[0]) # needs image height
    
    thickness_mul = 1.5 # staff_thickness_multiplier
    
#     rm_s_l = remove_staff_lines(img, runs, line_thickness, thickness_mul)
#     path_regular = './images/e/k{}_{}_b_{}_c_{}_RM_S_L.jpg'
#     cv2.imwrite(path_regular.format(k_w, k_h, block, c), rm_s_l)
    
    rm_s_l_up_down = rm_staff_lines_up_down_neighbours(img, runs, line_thickness, line_spacing, thickness_mul)
    path_up_down = './images/e/b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down.format(block, c), rm_s_l_up_down)
    
    path_up_down_dil = './images/e/DIL_3_1_x2_b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    up_down_dilated = cv2.dilate(rm_s_l_up_down, kernel)
    cv2.imwrite(path_up_down_dil.format(block, c), up_down_dilated)
    
    k_w = 5 # above kernel is 5 1. This is 5 2
    k_h = 2
    kernel = np.ones((k_w, k_h), dtype=np.int)
    path_up_down_dil = './images/e/DIL_5_2_b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    up_down_dilated = cv2.dilate(rm_s_l_up_down, kernel)
    cv2.imwrite(path_up_down_dil.format(block, c), up_down_dilated)
    
#     distance = int(line_spacing * 0.5) ### check for: * 1, * 1.3, * 1.5
#     rm_s_l_side = rm_staff_lines_side_neighbours(img, runs, line_thickness, line_spacing, thickness_mul, distance)
#     path_side = './images/e/b_{}_c_{}_RM_S_L_SIDE.jpg'
#     cv2.imwrite(path_side.format(block, c), rm_s_l_side)
# end


Top 4 most common black runs: [(3, 44701), (4, 22766), (2, 6691), (1, 3474)]
>>> line thickness:   3.5
Top 4 most common white runs [(21, 29309), (22, 11747), (20, 6365), (23, 3600)]
line spacing:  21.5


In [26]:
def dilate_and_save(img, kernel_w, kernel_h):
    '''Dilates a binary image with kernel of specified dimensions.
    saves the image to hard drive and returns the saved image'''
    kernel = np.ones((kernel_w, kernel_h), dtype=np.int)
    lines_only_img = cv2.dilate(img, kernel, iterations=1)
    lines_only_img_path = './images/locate_lines/dil_{}_{}.jpg'.format(kernel_w, kernel_h)
    cv2.imwrite(lines_only_img_path, lines_only_img)
    return lines_only_img

def erode_and_save(img, kernel_w_e, kernel_h_e, kernel_w_d=0, kernel_h_d=0):
    '''Erodes a binary image with kernel of specified dimensions.
    saves the image to hard drive and returns the saved image'''
    if (kernel_w_d == 0):
        dil_str = ''
    else:
        dil_str = '_dil_{}_{}'.format(kernel_w_d, kernel_h_d)
    
    kernel = np.ones((kernel_w_e, kernel_h_e), dtype=np.int)
    lines_only_img = cv2.erode(img, kernel, iterations=1)
    lines_only_img_path = './images/locate_lines/er_{}_{}{}.jpg'.format(kernel_w_e, kernel_h_e, dil_str)
    cv2.imwrite(lines_only_img_path, lines_only_img)
    return lines_only_img

def locate_lines(img):
    '''Locate lines.
    Returns: staff lines LOCATIONS and the lines-only image'''
    threshold = cv2.ADAPTIVE_THRESH_MEAN_C
    method, block, c = cv2.THRESH_BINARY, 55, 9
    img_ada = cv2.adaptiveThreshold(img, 255, threshold, method, block, c)

    # dilate with 1x50 --> erode with 2x50 
    kernel_w, kernel_h = 1, 50
    dilated = dilate_and_save(img_ada, kernel_w, kernel_h)
    kernel_w_e, kernel_h_e = 2, 50
    erode_the_dilated = erode_and_save(dilated, kernel_w, kernel_h)
#     erode_the_dilated = erode_and_save(dilated, kernel_w_e, kernel_h_e, kernel_w, kernel_h )
#     line locations = locate_lines # CONTINUE FROM TXT FILE !!!
#     line locations = locate_lines # CONTINUE FROM TXT FILE !!!
#     line locations = locate_lines # CONTINUE FROM TXT FILE !!!
    
img = cv2.imread('./images/dataset/muzikanti.JPG', 0) # 0 --> read as grayscale
locate_lines(img)